# Notebook Orbita Terrestre e Inserimento Orbita di Trasferimento Luna

## Approssimativamente basato sul piano di volo Apollo
### Notevolmente semplificato

Assunzione di orbite perfette e sempre dominate da uno dei due corpi

In [5]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Costanti fondamentali assunte
- Terra:
    - Re = 6371 km
    - Me = 5.97e24 kg
    
- Luna:
    - Rm = 1737 km
    - Mm = 7.34e22 kg
    - am = 384399 km
    - We assume circular orbit
    
- Low Earth Orbit altitude
    - hleo = 167 km  (altitude of the LEO above Earth surface)
    - hcsm = 110 km  (altitude of the CSM lunar orbit above Moon surface)

In [6]:
Re = 6371e3 # m
Me = 5.97e24 # kg
Rm = 1737e3 # m
Mm = 7.34e22 # kg
am = 384399e3 # m
G = 6.67e-11 # N m^2/kg
hleo = 167e3 # m
Rleo = Re+hleo
hcsm = 110e3 # m
Rcsm = Rm+hcsm
mue = Me*G
mum = Mm*G

In [11]:
class CircularOrbit(object):
    
    def __init__(self, mu, r, m=11000.):
        #
        # note: m is the mass of the orbiting object in kg
        #
        self.mu = mu
        self.r = r
        self.m = m
        self.E = self.Energy()
        self.P = self.Period()
        self.v = self.CircularVelocity()
        self.vesc = self.EscapeVelocity()
        
    def Energy(self):
        return -self.mu*self.m/self.r/2.0
        
    def Period(self):
        return np.sqrt(4.0*np.pi*np.pi*self.r*self.r*self.r/self.mu)
        
    def CircularVelocity(self):
        return np.sqrt(self.mu/self.r)
        
    def EscapeVelocity(self):
        return np.sqrt(2.0*self.mu/self.r)
        
class TransferOrbit(CircularOrbit):
    
    def __init__(self, mu, ro, a, m=11000.):
        self.mu = mu
        self.ro = ro
        self.a = a
        self.m = m
        CircularOrbit.__init__(self, self.mu, self.ro, m=self.m)
        self.v = self.Velocity(ro)
        
    def Energy(self):
        return -self.mu*self.m/(2.0*self.a)
    
    def Period(self):
        return np.sqrt(4.0*np.pi*np.pi*self.a*self.a*self.a/self.mu)
    
    def Velocity(self, r):
        return np.sqrt(2.0*(self.E/self.m+self.mu/r))

In [12]:
leo = CircularOrbit(mue, Rleo)
moon = CircularOrbit(mue, am)
tli = TransferOrbit(mue, Rleo, (Rleo+am)/2.)
csm = CircularOrbit(mum, Rcsm)

In [16]:
print("Vleo = {0} km/h  Pleo={1} min   Vesc = {2} km/h".format(leo.v/1.e3*3600,leo.P/60.,leo.vesc/1.e3*3600))
print("Vmoon = {0} km/h  Pmoon={1} days".format(moon.v/1.e3*3600,moon.P/86400.))
print("Vtli = {0} km/h  Ptli={1} days".format(tli.v/1.e3*3600,tli.P/86400./2.))
print("Vcsm = {0} km/h  Pcsm={1} min   Vesc = {2} km/h".format(csm.v/1.e3*3600,csm.P/60.,csm.vesc/1.e3*3600))

Vleo = 28095.070431722983 km/h  Pleo=87.729551641108 min   Vesc = 39732.42964036997 km/h
Vmoon = 3664.050567195556 km/h  Pmoon=27.465620636243305 days
Vtli = 39398.7877762471 km/h  Ptli=4.97967750825809 days
Vcsm = 5861.11172426256 km/h  Pcsm=118.80043044722032 min   Vesc = 8288.863691036067 km/h
